# ResNet YOLO Combination Demonstrational Notebook
## By: Matthew I. Swindall

### Load necessary libraries, models, and inintialize parameter variables

In [1]:
import os
import gc
import cv2
import torch
import random
import threading
import numpy as np
import pandas as pd
from PIL import Image
from PIL import ImageFile
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

# Allows PIL to load truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Appearance parameters for cv2 rectangles
color = (255, 0, 0)
thickness = 4

# Ancient Lives classes
classes = ['Alpha', 'Beta', 'Chi', 'Delta', 'Epsilon', 'Eta', 'Gamma', 'Iota', 'Kappa', 'Lambda', 'Mu', 'Nu', 'Omega', 'Omicron', 'Phi', 'Pi', 'Psi' ,'Rho', 'LunateSigma', 'Tau', 'Theta', 'Upsilon', 'Xi', 'Zeta']

# Load yolo segmentation model and KLD inference model
#seg = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt') # OLD VERSION TRAINED ON HOMER
seg = torch.hub.load('ultralytics/yolov5', 'custom', path='static/models/best_yolo.pt') # NEW VERSION TRAINED ON AL-ALL


# Load ResNet inference model
infer = load_model("static/models/alv2_cxe")

%matplotlib inline

C:\Users\matth\anaconda3\envs\flaskenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in C:\Users\matth/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-20 Python-3.7.16 torch-1.13.1+cpu CPU

Fusing layers... 


requirements: C:\Users\matth\anaconda3\envs\flaskenv\Lib\site-packages\requirements.txt not found, check failed.


Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


### Define functions for segmentation, inferencing, and displaying results

In [2]:
# Function uses yolo model to find characters in image of papyrus fragments
def Segment(img_path):
    im = cv2.imread(img_path)[..., ::-1]
    #im = Image.open(img_path)#[..., ::-1]
    results = seg(im, size=640)
    df = results.pandas().xyxy[0]
    df.rename(columns = {'confidence':'yolo_confidence'}, inplace = True)
    #print(df)
    return df

# Function uses Ancient Lives KLD ResNet model to crop characters and run multi-class inference on individual characters
def Inference(data, img_path, conf):
    pred_dist = []
    class_pred = []
    class_numeric = []
    res_prob = []
    df = data
    dfred = df[df['yolo_confidence'] > conf]
    im = cv2.imread(img_path)
    plt.imshow(im[..., ::-1])
    plt.title("Original Image")
    h, w, c = im.shape
    for i in range(len(dfred)):
        xmin = int(dfred['xmin'][i])
        xmax = int(dfred['xmax'][i])
        ymin = int(dfred['ymin'][i])
        ymax = int(dfred['ymax'][i])
        sp = (xmin, ymin)
        ep = (xmax, ymax)
        #print(sp, ep)
        #print("Domain: ", xmin, "-", xmax, "Range: ", ymin, "-", ymax)
        imgc = im.copy()
        imgc2 = im.copy()
        imgr = cv2.rectangle(imgc, sp, ep, color, thickness)[..., ::-1]
        cropped = imgc2[ymin:ymax, xmin:xmax][..., ::-1]
        res = cv2.resize(cropped, (70, 70), interpolation=cv2.INTER_AREA)
        x = []
        x.append(res)
        X = np.array(x)
        preds = infer.predict(X, verbose=0)
        imclass = np.argmax(preds[0])
        char = classes[imclass]
        prob = preds[0][imclass]
        pred_dist.append(preds)
        res_prob.append(prob)
        class_pred.append(char)
        class_numeric.append(imclass)
        ax2title = "Prediction:" + str(char) + " " + str(round((prob*100), 2)) + "% "
        fig, (ax1, ax2) = plt.subplots(1, 2)
        outname = "Prediction_" + str(i) + ".png"
        ax1.imshow(imgr)
        ax2.imshow(res)
        ax1.set_title("Location on Tablet")
        ax2.set_title(ax2title)
        plt.gcf()
        #plt.savefig(outname)
        plt.show()
    df["Predicted_Character"] = class_pred
    df["Predicted_Numeric"] = class_numeric
    df["Resnet_Confidence"] = res_prob
    
    
    csvname = "E:/homer_repo/homerres/" + img_path.split("/")[-1][:-4] + ".csv"
    #print(csvname)
    #print(df.head())
    #df.to_csv(csvname)
    
    
# Main function combining segmentation and inference
def Transcribe(img_path, conf):
    csvname = img_path.split("/")[-1][:-4] + ".csv"
    if not os.path.exists(csvname):
        output = Segment(img_path)
        Inference(output, img_path, conf)
    else:
        print(csvname, " ALREADY EXISTS")


### Get list of sample images

In [3]:
impath = "samples/"
frags = os.listdir(impath)
frags

['deadsea.jpg',
 'oxy1.jpg',
 'oxy2.jpg',
 'tablet1.jpg',
 'tablet2.jpg',
 'tablet3.jpg',
 'tablet4.png',
 'Z_POxy.v0015.n1800.a.05_135556.jpg']

### Perform transcription on selected image with YOLO confidence threshold value

In [ ]:
Transcribe(impath+frags[2], 0.0)